In [1]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.models import load_model
from keras import backend as K
import numpy as np
import random
import sys
import io

Using TensorFlow backend.
/home/imprs/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/imprs/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/imprs/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/imprs/anaconda3/lib/python3.6/site-packages/tensorflo

In [2]:
text = io.open('resources/shakespeare.txt', encoding='utf-8').read().lower()

print('corpus length:', len(text))

corpus length: 4030190


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))

total chars: 58


In [4]:
seqlen = 40 # Sequence length parameter
step = 5   # Determines the how many characters the window should be shifted in the text 
sequences = []  # List of sequences
char_class = [] # Corresponding class of each sequence

for i in range(0, len(text) - seqlen, step):
    sequences.append(text[i: i + seqlen])
    char_class.append(text[i + seqlen])
print('#no sequences:', len(sequences))

#no sequences: 806030


In [5]:
for idx in range(len(sequences[:10])):
    print(sequences[idx], ":" , char_class[idx])

1609

the sonnets

by william shakespear : e

the sonnets

by william shakespeare



 :  
sonnets

by william shakespeare



      :  
ts

by william shakespeare



           :  
y william shakespeare



                :  
liam shakespeare



                     :  
shakespeare



                     1
   : f
speare



                     1
  from  : f
e



                     1
  from faire : s
                     1
  from fairest cr : e


In [6]:
# Print from 1st to 10th character 
chars[:10]

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-']

In [7]:
# Indexed characters as dictionary
char_indices = dict((c, i) for i, c in enumerate(chars))

# Both matrices will initialized with zeros
training_set = np.zeros((len(sequences), seqlen, len(chars)), dtype=np.bool)
target_char = np.zeros((len(sequences), len(chars)), dtype=np.bool)
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        training_set[i, t, char_indices[char]] = 1
    target_char[i, char_indices[char_class[i]]] = 1

In [8]:
training_set.shape

(806030, 40, 58)

In [9]:
target_char.shape

(806030, 58)

In [10]:
model = Sequential()

#LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)

model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               95744     
_________________________________________________________________
dense_1 (Dense)              (None, 58)                7482      
_________________________________________________________________
activation_1 (Activation)    (None, 58)                0         
Total params: 103,226
Trainable params: 103,226
Non-trainable params: 0
_________________________________________________________________


In [11]:
def getNextCharIdx(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [12]:
# Creation of reverse char index, to get the char for the predicted class
indices_char = dict((i, c) for i, c in enumerate(chars))

def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    start_index = random.randint(0, len(text) - seqlen - 1)
    for diversity in [1, 0.1, 0.5]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = getNextCharIdx(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [13]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [14]:
model.fit(training_set, target_char,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
806016/806030 [============================>.] - ETA: 0s - loss: 1.6733
----- Generating text after Epoch: 0
----- diversity: 1
----- Generating with seed: "nd arviragus

  belarius. you, polydore,"
nd arviragus

  belarius. you, polydore, my bords, and all welcome,
    well versing his die not lave melen,
    weepedtipt in th' servance. i could proper.
  nos. ne le-ing so't's no ene?
  couried my lord.
    betwoen in the setten french in the di,
    terk, be wordall; bed.
  glord. feenie hove comeatice, as he know i formestzoves.
  halooucd. fair here leaves honour, ewen edfull.
                    exit where now
    come as good be disbought sild os my  
    not obporrnw to the tomber hunch to duken. i must do wast,
    i deever wownse may not bawr is lost come
    as say goven'd enough strangerritus,
    woa, you well; you we'll herefore, shall he heer notmad
    tabkingh, my hunies in the on 'o barder
    from a unpegue, hereful. aftece and own hopely brigely
    whus an

  carda, friend duke of swift, fishour., leal fight
    it, naglian timas leave me oft not entate.
                                          ange knows his.
  pear. if doth men doy, even well cannot by this naits,
    ace enobour vilage. so man's sepeat.
antibrao. 
----- diversity: 0.1
----- Generating with seed: "suffolk

  norfolk. well met, my lord ch"
suffolk

  norfolk. well met, my lord charge to the complete when he will see my sent to the father and man and sent the combony.
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

/home/imprs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


u.
  dog. a granto.
  well. o. in this quink the seld benemaed, the buturess.
  bartard are gracinent. o, and you are a thing mls; it will gs; that's behamk fuither, and thy laldey gomblesue
    french misswing and shall miss, and parting all crown!

                enter call it how ye will vilegn your greatdess;
    fears. why, sir-alof of nighing dursiay dirstm need, my father- sirray,
    should here mights he gain ans i nothing loves;
    we are rigs hands we. well wead, i'll still a  
  case! ay, then his down how in onfest'ste;
    parts my loon and his head it pridely of daughter.
                                            exit further.                                             
----- diversity: 0.1
----- Generating with seed: "d. away, then, with good courage!
    ye"
d. away, then, with good courage!
    yet the strain the court heart the theme the sense of the send the send the send the send the send the strain the sense.
                                                  

    to the prince of the bear the to the seem and the send the words and the bear the bear the come in the send the seem and this and the seem and this best the prince of the bear the man the bear the words and this be and this and the to the seem and see the seem and the world
    to the man that the bear the beat the words to the prince of the world
    that the beat of this make the man the company
    that the prince to the company thanks that the bear the world the common thanks and this to the seem and the for the best the prince of the come in the borne the bear the bellow and the bear the belliant thanks and the common thanks and the seem and see the father;
    and the beat the barth that the bear the world the company
    the world to the barth that the bear the send the bear the bardon and this to the send the for the duke of this best the come in the come to the co
----- diversity: 0.5
----- Generating with seed: "noble housewife with the time,
    to en"
noble housewife wi

    to put a strange face on his own the sens the bertime
    bedf the prince of soul was his into my lord.
    been are of the storns all of the princes.
  coronatio. why, and you were he shall we see his lord of my duty
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [16]:
model.save('shakespeareModel1.h5')